In [3]:
# %% [markdown]
# OCR and LLM Timing Analysis Notebook
# Этот ноутбук измеряет время обработки OCR и LLM запросов на образцах изображений из папки `samples`.

# %%
# Импорт необходимых библиотек и установка окружения
import os
import time
from pathlib import Path

from PIL import Image
import pytesseract

# Optional: укажите путь к tesseract.exe на Windows, если он не в PATH
if os.name == "nt":
    _tesseract_path = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    if Path(_tesseract_path).exists():
        pytesseract.pytesseract.tesseract_cmd = _tesseract_path

# Импорт функции препроцессинга (предполагается, что preprocessing.py лежит рядом с ноутбуком)
try:
    from preprocessing import preprocess_image
except ImportError:
    raise ImportError("Убедитесь, что файл preprocessing.py находится в рабочей папке")

# Импорт модели LLM из LangChain
try:
    from langchain_community.chat_models import ChatOpenAI  # langchain >=0.2.0
except ImportError:
    from langchain.chat_models import ChatOpenAI  # langchain <0.2.0

from langchain.schema import SystemMessage, HumanMessage

# Загрузка переменных окружения
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise RuntimeError("Установите OPENAI_API_KEY в переменных окружения или в .env файле")

# %%
# Определение функций для OCR и запроса к LLM

def image_to_text(image_path: Path) -> str:
    """
    Выполняет препроцессинг, затем OCR оригинального изображения.
    """
    try:
        _ = preprocess_image(image_path)
    except Exception as e:
        print(f"Препроцессинг не удался для {image_path.name}: {e}")
    with Image.open(image_path) as im:
        return pytesseract.image_to_string(im)


def ask_llm(source_text: str, question: str) -> str:
    """
    Отправляет вопрос LLM, основываясь строго на тексте source_text.
    """
    llm = ChatOpenAI(
        model_name="gpt-4o",
        temperature=0,
        openai_api_key=OPENAI_API_KEY,
    )
    messages = [
        SystemMessage(
            content=(
                "Ты — помощник, который отвечает на вопросы по тексту, извлечённого из документа. "
                "Текст может содержать ошибки распознавания — если это случается, додумывай контекст. "
                "Отвечай ТОЛЬКО на основании предоставленного текста. Если ответа нет, скажи, что не знаешь."
            )
        ),
        HumanMessage(content=f"Текст:\n{source_text}\n\nВопрос: {question}"),
    ]
    return llm(messages).content.strip()

# %%
# Измерение времени OCR и LLM для изображений из папки samples
sample_dir = Path("samples")
results = []
question = "Напиши краткий обзор текста."  # Можно изменить вопрос

print(f"Обработка {sample_dir} (всего {len(list(sample_dir.glob('*.png'))) + len(list(sample_dir.glob('*.jpg'))) + len(list(sample_dir.glob('*.jpeg')))} файлов)")

for img_path in sorted(sample_dir.glob("*.png")) + sorted(sample_dir.glob("*.jpg")) + sorted(sample_dir.glob("*.jpeg")):
    # OCR
    ocr_start = time.time()
    text = image_to_text(img_path)
    ocr_time = time.time() - ocr_start
    # LLM
    llm_start = time.time()
    answer = ask_llm(text, question)
    llm_time = time.time() - llm_start
    # Сохраняем результат
    results.append((img_path.name, ocr_time, llm_time, answer))
    print(f"{img_path.name}: OCR {ocr_time:.2f}s | LLM {llm_time:.2f}s")

# %%
# Вывод сводной таблицы
print("\nРезультаты обработки:")
print(f"{'Название документа':<30} | {'OCR (сек)':>10} | {'LLM (сек)':>10} | Ответ LLM")
print("-" * 80)
for name, ocr_t, llm_t, ans in results:
    # Ограничение длины ответа для вывода (например, первые 60 символов)
    short_ans = ans.replace("\n", " ")[:60] + ("..." if len(ans) > 60 else "")
    print(f"{name:<30} | {ocr_t:10.2f} | {llm_t:10.2f} | {short_ans}")

Обработка samples (всего 12 файлов)
8txbX.png: OCR 0.57s | LLM 1.91s
document-screenshot.png: OCR 0.71s | LLM 2.85s
george.png: OCR 0.39s | LLM 1.71s
scanned_document.png: OCR 0.76s | LLM 2.19s
1_eTJ5_mezfc-K3WKyxpls8Q.jpg: OCR 0.69s | LLM 2.65s
f8GOA.jpg: OCR 0.42s | LLM 3.01s
iiif-public_gdcmassbookdig_libertyuniononei00newy_libertyuniononei00newy_0007-full-pct_25.0-0-default.jpg: OCR 0.32s | LLM 2.80s
language_of_law.05-8135ca3beafb99acbd5cfcc85c3340211fff4e1167c8ccfd037ace01be45847b.jpg: OCR 0.36s | LLM 1.42s
language_of_law.06-331b4b9c94172fa0d20877f28e585fbd629ee8c0b30ea1b20c98f3ebd107d505.jpg: OCR 0.17s | LLM 1.01s
librarydeed1.jpg: OCR 5.46s | LLM 2.73s
owenbrown.jpg: OCR 7.42s | LLM 2.76s
scanned_jpg_to_docx.jpg: OCR 0.71s | LLM 3.65s

Результаты обработки:
Название документа             |  OCR (сек) |  LLM (сек) | Ответ LLM
--------------------------------------------------------------------------------
8txbX.png                      |       0.57 |       1.91 | Текст обсуждае

In [6]:
import pandas as pd
df = pd.DataFrame(results, columns=["Название документа", "OCR (сек)", "LLM (сек)", "Ответ LLM"])
df.to_excel("time_check.xlsx", index=False)